<img src="res/Manchester_United_F.C.-Logo.wine.png"
    style="width:400px; float: right; margin: 0 40px 40px 40px;"></img>

# 2. Xử lý dữ liệu

Làm sạch những dữ liệu thô đã thu thập được từ bước phía trước

![purple-divider](res/purple.png)
## Cài đặt

In [223]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


![green-divider](res/green.png)
## Nhập dữ liệu

In [421]:
import pandas as pd
import ast
import re
from datetime import timedelta, datetime

In [422]:
df = pd.read_csv("data/facebook_data_raw.csv")
df.shape

(500, 52)

![green-divider](res/green.png)
## Xử lý dữ liệu
#### Xử lý dữ liệu quan trọng bị thiếu
- Lấy trường post_id làm khóa, lọc bỏ dữ liệu bị trùng lặp
- Xóa những hàng bị thiếu dữ liệu ở các cột quan trọng (**post_id**, **reactions**, **comments**, **shares**, **time**, **post_text**, **comments_full**)
- Những hàng bị thiếu sẽ được lưu vào **missing_posts.txt** để đi thu thập lại

In [423]:
df.drop_duplicates(subset='post_id', inplace = True)

In [424]:
cols = ["post_id", "reactions", "comments", "shares", "time", "post_text", "comments_full"]
mask = df[cols].isna().any(axis=1)
posts_to_drop = df[mask]

df.dropna(subset=cols, inplace=True)
len(posts_to_drop)

0

In [425]:
with open("data/missing_posts.txt", mode="w") as file:
    for post_id in posts_to_drop.post_id:
        file.write("https://facebook.com/ManUnitedVN1878/posts/" + post_id + "\n")
print("Done")

Done


#### Lấp đầy những chỗ dữ liệu bị thiếu, tạo thêm những cột cần thiết
- **reactions** sẽ được bổ sung những cảm xúc bị thiếu với giá trị là 0
- **reactions_count** bị thiếu sẽ được tính lại từ **reactions**
- Bổ sung **has_image**, **has_video**
- Bổ sung **comments_text** (chứa một chuỗi gồm các bình luận trong bài viết) từ trường **comments_full**
- Bổ sung **commenters_url** (chứa link dẫn đến facebook những người đã bình luận bài viết) từ trường **comments_full**
- Bổ sung **reactors_url** (chứa link dẫn đến facebook những người đã thả cảm xúc bài viết) từ trường **reactors**
- Bổ sung **engagement_summary** là tổng số lượt tương tác
- Tách **date** từ **time**
- **time** mới chỉ chứa giờ, phút, giây
- Sắp xếp dữ liệu theo thời gian tăng dần

In [426]:
df['reactions'] = df['reactions'].apply(ast.literal_eval)

reactions = ["thích", "yêu thích", "thương thương", "haha", "wow", "buồn", "phẫn nộ"]
for i in range(len(df.reactions)):
    for key in reactions:
        if key not in df.iloc[i].reactions:
            df.iloc[i].reactions[key] = 0

In [427]:
reactions_count_nan = df[df.reaction_count.isna()]
for i in reactions_count_nan.index:
    df.at[i, "reaction_count"] = sum(df.iloc[i].reactions.values())

In [428]:
df["has_image"] = df.image.notna()
df["has_video"] = df.video.notna()

df['images'].fillna("", inplace = True)
df['images'] = df['images'].apply(ast.literal_eval)

In [429]:
def extract_comment_text(input_string):
    pattern = r"comment_text\': \'(.*?)\', \'comment_time"
    matches = re.findall(pattern, input_string)
    if matches:
        return '  '.join(matches)         
    else:
        return " "

df['comments_text'] = df['comments_full'].apply(extract_comment_text)

In [430]:
def extract_commenters_url(input_string):
    pattern = r"commenter_url\': \'(.*?)\', \'commenter_name"
    matches = re.findall(pattern, input_string)
    if matches:
        return matches
    else:
        return []

df['commenters_url'] = df['comments_full'].apply(extract_commenters_url)

In [431]:
def extract_reactors_url(input_string):
    pattern = r"\'link\': \'(.*?)\', \'type"
    matches = re.findall(pattern, input_string)
    if matches:
        return matches
    else:
        return []
    
df['reactors_url'] = df["reactors"].apply(extract_reactors_url)

In [432]:
df['engagement_summary'] = df['reaction_count'] + df['shares'] + df['comments']

In [433]:
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df['time'] = df['time'].dt.strftime('%H:%M:%S')
df.sort_values(by=["date", "time"], inplace = True)

![green-divider](res/green.png)
### Lọc những ngày bị thiếu dữ liệu
Lọc ra những ngày không có bất kì bài viết nào.
Mục đích là làm cho dữ liệu liên tục về mặt thời gian

In [434]:
s1 = df.iloc[0].date.strftime('%Y-%m-%d')
s2 = df.iloc[-1].date.strftime('%Y-%m-%d')

start_date = datetime.strptime(s1, '%Y-%m-%d')
end_date = datetime.strptime(s2, '%Y-%m-%d')

date_list = []

while start_date <= end_date:
    date_list.append(start_date.strftime('%Y-%m-%d'))
    start_date += timedelta(days=1)

df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

c = list(set(date_list) - set(df["date"]))
c.sort()
c

['2023-08-13',
 '2023-08-15',
 '2023-08-16',
 '2023-08-17',
 '2023-08-18',
 '2023-08-19',
 '2023-08-20',
 '2023-08-21',
 '2023-08-22',
 '2023-08-23',
 '2023-08-24',
 '2023-08-25',
 '2023-08-26',
 '2023-08-27',
 '2023-08-28',
 '2023-08-29',
 '2023-08-30',
 '2023-09-01',
 '2023-09-02',
 '2023-09-03',
 '2023-09-04',
 '2023-09-05',
 '2023-09-06',
 '2023-09-07',
 '2023-09-08',
 '2023-09-09',
 '2023-09-10',
 '2023-09-11',
 '2023-09-12',
 '2023-09-13',
 '2023-09-14',
 '2023-09-15',
 '2023-09-16',
 '2023-09-17',
 '2023-09-18']

![green-divider](res/green.png)
## Xử lý để dữ liệu liên tục

Chúng ta thấy, dữ liệu trên bị thiếu 35 ngày liền.\
Sau khi kiểm tra thì thực sự không thể tìm thấy bất kì bài viết nào trong khoảng thời gian 35 ngày đó.\
Cách khắc phục là chỉ lấy đoạn dữ liệu liên tục gần với hiện tại nhất.

In [435]:
z = datetime.strptime(c[-1], '%Y-%m-%d') + timedelta(days=1)
z = z.strftime('%Y-%m-%d')

z = df[df.date == z].index[0]
# z = df.index.get_loc[df.index[z]]
df.index[z]
df = df[209:]
df

,original_request_url,post_url,post_id,text,post_text,shared_text,original_text,time,timestamp,image,...,image_ids,was_live,fetched_time,has_image,has_video,comments_text,commenters_url,reactors_url,engagement_summary,date
290,https://facebook.com/ManUnitedVN1878/posts/721...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02op7JrbwsDi5goA1kXg1j3kfrCqNPY6UzRsQ5bhd...,"❌ 4 mùa đầu tiên của Klopp: Xếp thứ 8️⃣, 4️⃣, ...","❌ 4 mùa đầu tiên của Klopp: Xếp thứ 8️⃣, 4️⃣, ...",NaN,NaN,10:20:17,1.695094e+09,https://scontent.fhan5-11.fna.fbcdn.net/v/t39....,...,['721037750066904'],False,2023-11-18 13:35:18.740830,True,False,vào đây này mua bán đồ đẹp rẻ GÓC THANH LÝ QUẦ...,[https://facebook.com/profile.php?id=615500206...,[https://facebook.com/profile.php?id=100075493...,116.0,2023-09-19
289,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02wAnyURhLMmBBQsGqzKGEuKUq35dHucsWp3df7Fy...,80' Bayern Munich 2-0 Man United\n.\n.\n.\n\nN...,80' Bayern Munich 2-0 Man United\n.\n.\n.\n\nN...,NaN,NaN,23:12:21,1.695226e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722063183297694'],False,2023-11-18 13:34:51.236238,True,False,Báo trước r 🥲 Fan mu nhưng vẫn đoán mu thua 4...,[https://facebook.com/profile.php?id=100044849...,[https://facebook.com/profile.php?id=100095215...,593.0,2023-09-20
288,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02DACwihodWJbEWg61nGNP2PfYUGFkyeCeEMr41RE...,📸 Đội hình ra sân của Man United với Bayern Mu...,📸 Đội hình ra sân của Man United với Bayern Mu...,NaN,NaN,01:01:31,1.695233e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722121259958553'],False,2023-11-18 13:34:32.178678,True,False,choi thế này thì xin chúc mừng đã thắng trong ...,[https://facebook.com/profile.php?id=100015388...,[https://facebook.com/profile.php?id=100093706...,199.0,2023-09-21
287,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0QKc4TLmnK8ACBsCAVEMwpdzUjkCjhyLeQ2HWx2pj...,Anh em đâu hết rồi,Anh em đâu hết rồi,NaN,NaN,02:05:42,1.695237e+09,https://scontent.fhan5-6.fna.fbcdn.net/v/t39.3...,...,['722150159955663'],False,2023-11-18 13:34:13.961985,True,False,Thiến thằng trọc này đi tụi bay🤣 Cút đi Ông ...,[https://facebook.com/manh.lyvan.50951?eav=AfY...,[https://facebook.com/profile.php?id=100089589...,191.0,2023-09-21
286,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0LLdHFw9YecH2A3JpGLSH3JM1Z3CEytTQZWbEJwX2...,Sane mở tỷ số cho Bayern\n\nMan United 0-1 Bayern,Sane mở tỷ số cho Bayern\n\nMan United 0-1 Bayern,NaN,NaN,02:30:48,1.695238e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722158196621526'],False,2023-11-18 13:33:58.131353,True,False,Ko biết nói gì😢 Thầy 10 khó đúng là có mắt nh...,[https://facebook.com/hop.nguyen.50746444?eav=...,[],183.0,2023-09-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,https://facebook.com/ManUnitedVN1878/posts/755...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0wAuNKrMGimqisiGStw3SDiy6RBTLuL9Lc8AWzA7D...,Man United sẽ có 130 củ Euro nếu bán Casemiro ...,Man United sẽ có 130 củ Euro nếu bán Casemiro ...,NaN,NaN,10:21:28,1.700105e+09,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,...,['755446253292720'],False,2023-11-17 21:59:36.415970,True,False,Kha khá hay ko thì phải biết mua. Chứ bế thánh...,[https://facebook.com/nhi.ka.129?eav=Afaf5HCv1...,[],824.0,2023-11-16
3,https://facebook.com/ManUnitedVN1878/posts/755...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02VsE9ujvo677K9rrYEiK7GRxw3m115xppjJ4GQto...,"Nói về đồ nhiệt huyết, tốc độ, kỹ thuật lẫn tu...","Nói về đồ nhiệt huyết, tốc độ, kỹ thuật lẫn tu...",NaN,NaN,19:55:13,1.700139e+09,https://scontent-hkg4-2.xx.fbcdn.net/v/t39.308...,...,['755675066603172'],False,2023-11-17 21:59:22.105178,True,False,mình có 3 người.. đần như nha

![green-divider](res/green.png)
## Những bài viết mới được đăng khi thu thập sẽ không phản ánh đúng được lượng tương tác mà chúng nên có.

Để tránh ảnh hưởng đến những bài viết khác, chúng ta sẽ xóa bớt đi 10 bài viết gần nhất.

In [436]:
df = df[:-10]
df

,original_request_url,post_url,post_id,text,post_text,shared_text,original_text,time,timestamp,image,...,image_ids,was_live,fetched_time,has_image,has_video,comments_text,commenters_url,reactors_url,engagement_summary,date
290,https://facebook.com/ManUnitedVN1878/posts/721...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02op7JrbwsDi5goA1kXg1j3kfrCqNPY6UzRsQ5bhd...,"❌ 4 mùa đầu tiên của Klopp: Xếp thứ 8️⃣, 4️⃣, ...","❌ 4 mùa đầu tiên của Klopp: Xếp thứ 8️⃣, 4️⃣, ...",NaN,NaN,10:20:17,1.695094e+09,https://scontent.fhan5-11.fna.fbcdn.net/v/t39....,...,['721037750066904'],False,2023-11-18 13:35:18.740830,True,False,vào đây này mua bán đồ đẹp rẻ GÓC THANH LÝ QUẦ...,[https://facebook.com/profile.php?id=615500206...,[https://facebook.com/profile.php?id=100075493...,116.0,2023-09-19
289,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02wAnyURhLMmBBQsGqzKGEuKUq35dHucsWp3df7Fy...,80' Bayern Munich 2-0 Man United\n.\n.\n.\n\nN...,80' Bayern Munich 2-0 Man United\n.\n.\n.\n\nN...,NaN,NaN,23:12:21,1.695226e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722063183297694'],False,2023-11-18 13:34:51.236238,True,False,Báo trước r 🥲 Fan mu nhưng vẫn đoán mu thua 4...,[https://facebook.com/profile.php?id=100044849...,[https://facebook.com/profile.php?id=100095215...,593.0,2023-09-20
288,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02DACwihodWJbEWg61nGNP2PfYUGFkyeCeEMr41RE...,📸 Đội hình ra sân của Man United với Bayern Mu...,📸 Đội hình ra sân của Man United với Bayern Mu...,NaN,NaN,01:01:31,1.695233e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722121259958553'],False,2023-11-18 13:34:32.178678,True,False,choi thế này thì xin chúc mừng đã thắng trong ...,[https://facebook.com/profile.php?id=100015388...,[https://facebook.com/profile.php?id=100093706...,199.0,2023-09-21
287,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0QKc4TLmnK8ACBsCAVEMwpdzUjkCjhyLeQ2HWx2pj...,Anh em đâu hết rồi,Anh em đâu hết rồi,NaN,NaN,02:05:42,1.695237e+09,https://scontent.fhan5-6.fna.fbcdn.net/v/t39.3...,...,['722150159955663'],False,2023-11-18 13:34:13.961985,True,False,Thiến thằng trọc này đi tụi bay🤣 Cút đi Ông ...,[https://facebook.com/manh.lyvan.50951?eav=AfY...,[https://facebook.com/profile.php?id=100089589...,191.0,2023-09-21
286,https://facebook.com/ManUnitedVN1878/posts/722...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0LLdHFw9YecH2A3JpGLSH3JM1Z3CEytTQZWbEJwX2...,Sane mở tỷ số cho Bayern\n\nMan United 0-1 Bayern,Sane mở tỷ số cho Bayern\n\nMan United 0-1 Bayern,NaN,NaN,02:30:48,1.695238e+09,https://scontent.fhan5-8.fna.fbcdn.net/v/t39.3...,...,['722158196621526'],False,2023-11-18 13:33:58.131353,True,False,Ko biết nói gì😢 Thầy 10 khó đúng là có mắt nh...,[https://facebook.com/hop.nguyen.50746444?eav=...,[],183.0,2023-09-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,https://facebook.com/ManUnitedVN1878/posts/753...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0VcV88qaV2gsPMuwMqqi83Kb53G7pMA3txjcn83WA...,"Các cựu danh thủ, cầu thủ và các quan chức CLB...","Các cựu danh thủ, cầu thủ và các quan chức CLB...",NaN,NaN,21:38:01,1.699886e+09,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,...,[],False,2023-11-17 22:02:01.235794,True,False,,[],[],238.0,2023-11-13
13,https://facebook.com/ManUnitedVN1878/posts/754...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid0232KzeWRiFecHkBrbyL1WBLZdzibN3SfwHe4YK1D...,Chính thức!!! Trang chủ thông báo Eriksen bị c...,Chính thức!!! Trang chủ thông báo Eriksen bị c...,NaN,NaN,08:33:04,1.699926e+09,https://scontent-hkg4-1.xx.fbcdn.net/v/t39.308...,...,['754244116746267'],False,2023-11-17 22:01:34.416401,True,False,Cơ hội cho thằng nhỏ Mainoo comeback 😄 Mịa nă...,[https://facebook.com/nguyen.duyhung.1232?eav=...,[],310.0,2023-11-14
12,https://facebook.c

![green-divider](res/green.png)
## Chia thành các loại phù hợp, bỏ những cột dư thừa
Dữ liệu đã xử lý ở trên không có bước lọc bỏ những trường dữ liệu không cần thiết bởi vì chúng ta sẽ chọn và chỉ lấy những trường cần thiết ở bước này.
- **df1** chứa những dữ liệu mà chúng ta dùng để phân tích.
- **df2** chứa đường link dẫn đến bài viết, ảnh, video,...

In [437]:
df1_columns = ["post_id", "post_text", "date", "time", "comments", "shares", "reactions", 
               "reaction_count", "has_image", "has_video", "comments_text", "engagement_summary"]   
df2_columns = ["post_id", "post_url", "images", "video", "commenters_url", "reactors_url"]

In [438]:
df1 = pd.DataFrame(df, columns=df1_columns)
df2 = pd.DataFrame(df, columns=df2_columns)

![green-divider](res/green.png)
## Lưu dữ liệu sau khi hoàn thành
Sau khi hoàn thành xử lý dữ liệu, từ 500 bài viết, chúng ta còn lại 281 bài trong khoảng thời gian từ 19/9/2023 đến 14/11/2023\
Lưu dữ liệu vào 2 dataframe vào 2 file **posts_data.csv** và **posts_urls.csv**

In [439]:
df1.to_csv("data/posts_data.csv", mode="w", index=False)
df2.to_csv("data/posts_urls.csv", mode="w", index=False)
print("Done")

Done


![purple-divider](res/purple.png)